### Pair Programming - ETL II

In [1]:
import requests
import pandas as pd
import numpy as np
import ast

from datetime import datetime, timedelta
pd.options.display.max_columns = None

Usando el csv de attacks_limpieza_completa y el csv con los datos climáticos del anterior pair. Deberemos juntarlos en una única fuente de información.



El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. 
Para ello:
- Cargaremos los dos ficheros de datos.

In [2]:
df_tiburones = pd.read_csv("datos/attacks_limpieza_completa.csv", index_col=0)

df_clima = pd.read_csv("datos/tiempo_paises_1.csv", index_col=0).reset_index()

In [3]:
df_clima.head(2)

,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,0,3,5,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",-2,15,12,1032,none,0,-9999,300,3,USA
1,1,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",3,15,7,1031,none,0,1,330,3,USA


In [4]:
df_tiburones.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860


El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. 
Para ello:
- Cargaremos los dos ficheros de datos
- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer: (USA, Australia, New Zealand, South Africa, Papua New Guinea)
- Del dataframe de los datos climáticos seleccionaremos todas las columnas.
- Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
- Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.
- Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. 
Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer: (USA, Australia, New Zealand, South Africa, Papua New Guinea)

In [5]:
paises = ["usa", "australia", "new zealand", "south africa", "papua new guinea"]

In [6]:
df_tiburones_country = df_tiburones[df_tiburones["country"].isin(paises)]
df_tiburones_country["country"].unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

In [7]:
# Como los datos de las columnas de países en un df está capitalizado y en el otro en minúsculas vamos a unificarlo un poco.

df_clima["country"] = df_clima["country"].str.lower()

In [8]:
df_clima.head(1)

,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,0,3,5,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",-2,15,12,1032,none,0,-9999,300,3,usa


In [9]:
df_tiburones_country.head(1)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327


In [10]:
# Procedemos a limpiar las columnas del df_clima de "rh_profile" y "wind_profile" 

df_clima['rh_profile'] = df_clima['rh_profile'].apply(ast.literal_eval)

In [11]:
df_rh = df_clima["rh_profile"].apply(pd.Series)
df_rh.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': -9999}","{'layer': '900mb', 'rh': 9}","{'layer': '850mb', 'rh': 6}","{'layer': '800mb', 'rh': 5}","{'layer': '750mb', 'rh': 3}","{'layer': '700mb', 'rh': 2}","{'layer': '650mb', 'rh': 0}","{'layer': '600mb', 'rh': 0}","{'layer': '550mb', 'rh': 1}","{'layer': '500mb', 'rh': 13}","{'layer': '450mb', 'rh': 16}","{'layer': '400mb', 'rh': 16}","{'layer': '350mb', 'rh': 16}","{'layer': '300mb', 'rh': 13}","{'layer': '250mb', 'rh': -2}","{'layer': '200mb', 'rh': -3}"


- Antes de hacer el groupby tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Separamos la información en columnas.

In [12]:
for i in range(len(df_rh.columns)):
    
    nombre = "rh_" + str(df_rh[i].apply(pd.Series)["layer"][0]) 

    valores = list(df_rh[i].apply(pd.Series)["rh"])

    df_clima.insert(i, nombre, valores)

In [13]:
df_clima.head(1)

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,-9999,9,6,5,3,2,0,0,1,13,16,16,16,13,-2,-3,0,3,5,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",-2,15,12,1032,none,0,-9999,300,3,usa


In [14]:
df_clima['wind_profile']= df_clima['wind_profile'].apply(ast.literal_eval)

In [15]:
df_wind= df_clima["wind_profile"].apply(pd.Series)
df_wind.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': -9999, 'speed'...","{'layer': '900mb', 'direction': 350, 'speed': 5}","{'layer': '850mb', 'direction': 350, 'speed': 5}","{'layer': '800mb', 'direction': 345, 'speed': 5}","{'layer': '750mb', 'direction': 335, 'speed': 5}","{'layer': '700mb', 'direction': 335, 'speed': 6}","{'layer': '650mb', 'direction': 340, 'speed': 5}","{'layer': '600mb', 'direction': 340, 'speed': 5}","{'layer': '550mb', 'direction': 335, 'speed': 5}","{'layer': '500mb', 'direction': 315, 'speed': 4}","{'layer': '450mb', 'direction': 295, 'speed': 5}","{'layer': '400mb', 'direction': 300, 'speed': 5}","{'layer': '350mb', 'direction': 295, 'speed': 5}","{'layer': '300mb', 'direction': 290, 'speed': 5}","{'layer': '250mb', 'direction': 325, 'speed': 6}","{'layer': '200mb', 'direction': 320, 'speed': 8}"


In [16]:
for i in range(len(df_wind.columns)):

    nombre = "wind_direction" + str(df_wind[i].apply(pd.Series)["layer"][0])
    
    valor1 = list(df_wind[i].apply(pd.Series)["direction"])
    
    df_clima.insert(i, nombre, valor1)


for i in range(len(df_wind.columns)):

    nombre2 = "wind_speed" + str(df_wind[i].apply(pd.Series)["layer"][0])

    valor2 = list(df_wind[i].apply(pd.Series)["speed"])

    df_clima.insert(i, nombre2, valor2)

In [17]:
df_clima.head(2)

,wind_speed950mb,wind_speed900mb,wind_speed850mb,wind_speed800mb,wind_speed750mb,wind_speed700mb,wind_speed650mb,wind_speed600mb,wind_speed550mb,wind_speed500mb,wind_speed450mb,wind_speed400mb,wind_speed350mb,wind_speed300mb,wind_speed250mb,wind_speed200mb,wind_direction950mb,wind_direction900mb,wind_direction850mb,wind_direction800mb,wind_direction750mb,wind_direction700mb,wind_direction650mb,wind_direction600mb,wind_direction550mb,wind_direction500mb,wind_direction450mb,wind_direction400mb,wind_direction350mb,wind_direction300mb,wind_direction250mb,wind_direction200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,-9999,5,5,5,5,6,5,5,5,4,5,5,5,5,6,8,-9999,350,350,345,335,335,340,340,335,315,295,300,295,290,325,320,-9999,9,6,5,3,2,0,0,1,13,16,16,16,13,-2,-3,0,3,5,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",-2,15,12,1032,none,0,-9999,300,3,usa
1,-9999,3,5,5,5,5,5,5,5,5,5,5,5,7,8,8,-9999,340,345,340,335,335,340,335,325,320,320,320,325,350,335,325,-9999,10,7,6,4,1,-1,1,5,10,9,11,12,1,0,-3,1,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",3,15,7,1031,none,0,1,330,3,usa


In [18]:
# Borramos las columnas originales que como ya están fuera no hacen falta.

df_clima = df_clima.drop(columns=["rh_profile", "wind_profile"], axis= 1)

- Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.

In [19]:
df_clima_medias = df_clima.groupby("country").mean().reset_index()

/tmp/ipykernel_4425/1277389351.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_clima_medias = df_clima.groupby("country").mean().reset_index()


In [20]:
df_clima_medias.head()

,country,wind_speed950mb,wind_speed900mb,wind_speed850mb,wind_speed800mb,wind_speed750mb,wind_speed700mb,wind_speed650mb,wind_speed600mb,wind_speed550mb,wind_speed500mb,wind_speed450mb,wind_speed400mb,wind_speed350mb,wind_speed300mb,wind_speed250mb,wind_speed200mb,wind_direction950mb,wind_direction900mb,wind_direction850mb,wind_direction800mb,wind_direction750mb,wind_direction700mb,wind_direction650mb,wind_direction600mb,wind_direction550mb,wind_direction500mb,wind_direction450mb,wind_direction400mb,wind_direction350mb,wind_direction300mb,wind_direction250mb,wind_direction200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,australia,3.968750,4.078125,3.750000,3.640625,3.609375,3.671875,3.890625,4.000000,3.437500,3.281250,3.109375,3.171875,4.109375,4.859375,5.593750,5.640625,100.62500,100.234375,103.750000,112.03125,122.109375,137.578125,148.828125,153.437500,151.718750,157.187500,169.765625,200.156250,229.843750,251.093750,260.390625,263.515625,1.234375,2.203125,3.406250,4.953125,7.156250,10.109375,11.406250,9.359375,2.546875,0.296875,-0.12500,-1.078125,-0.578125,-0.3750,-1.453125,-2.406250,31.5,97.5,3.546875,-9999.0,-9999.0,-9999.0,33.562500,2.187500,0.750000,1006.703125,0.234375,0.00000,102.578125,3.140625
1,new zealand,-2341.953125,2.343750,2.390625,2.515625,2.765625,2.656250,2.562500,2.718750,2.875000,3.093750,3.234375,3.625000,4.203125,4.765625,5.125000,5.000000,-2242.03125,168.750000,191.875000,205.78125,197.109375,185.000000,178.437500,179.843750,200.468750,193.046875,188.281250,178.828125,185.390625,180.078125,166.484375,168.828125,-2336.093750,10.046875,10.671875,10.468750,8.890625,3.656250,0.843750,0.296875,0.921875,0.812500,1.56250,2.109375,3.390625,5.6875,6.406250,2.500000,31.5,97.5,5.187500,-9999.0,-9999.0,-9999.0,15.343750,5.250000,9.906250,1017.031250,1.828125,0.00000,151.015625,2.031250
2,papua new guinea,-9999.000000,-9999.000000,-9999.000000,1.953125,1.984375,2.359375,2.671875,2.906250,2.953125,2.953125,3.593750,3.937500,4.125000,4.437500,4.781250,5.437500,-9999.00000,-9999.000000,-9999.000000,171.71875,209.296875,257.343750,247.265625,160.312500,119.687500,94.843750,62.421875,66.562500,99.609375,112.734375,110.234375,112.500000,-9999.000000,-9999.000000,-9999.000000,12.218750,12.765625,12.265625,12.031250,12.328125,12.156250,11.187500,11.03125,11.625000,13.218750,12.5000,11.562500,10.234375,31.5,97.5,8.687500,-9999.0,-9999.0,-9999.0,14.953125,-0.296875,13.421875,-9999.000000,4.296875,0.00000,205.703125,1.984375
3,south africa,-9999.000000,-9999.000000,3.390625,3.125000,2.796875,2.625000,2.625000,2.578125,2.734375,2.953125,3.156250,3.375000,3.015625,2.812500,2.921875,3.484375,-9999.00000,-9999.000000,173.593750,176.25000,176.640625,201.953125,193.984375,208.281250,183.671875,235.781250,274.765625,261.328125,253.281250,234.609375,237.734375,269.218750,-9999.000000,-9999.000000,0.796875,1.031250,1.953125,3.296875,5.453125,8.125000,7.718750,3.015625,1.28125,2.953125,4.343750,4.2500,6.125000,7.703125,31.5,97.5,5.812500,-9999.0,-9999.0,-9999.0,30.484375,1.125000,0.875000,1009.484375,1.656250,0.00000,187.968750,2.921875
4,usa,-9999.000000,-152.734375,4.015625,4.062500,4.062500,4.109375,4.265625,4.625000,4.984375,5.328125,5.828125,6.359375,7.046875,7.781250,8.781250,9.296875,-9999.00000,53.296875,229.453125,241.25000,248.125000,256.562500,261.953125,274.765625,273.125000,271.796875,268.437500,266.484375,259.531250,264.921875,264.218750,262.656250,-9999.000000,-148.375000,6.812500,5.312500,5.468750,5.515625,5.750000,6.250000,6.812500,7.078125,7.43750,9.140625,10.250000,10.0625,6.265625,2.515625,31.5,97.5,6.703125,-9999.0,-9999.0,-9999.0,2.281250,13.343750,9.281250,1009.593750,0.796875,-468.09375,210.468750,2.718750


In [21]:
# Procedemos a unir los dos DataFrames en uno.

df_total = df_tiburones_country.merge(df_clima_medias, on="country")
df_total.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM,wind_speed950mb,wind_speed900mb,wind_speed850mb,wind_speed800mb,wind_speed750mb,wind_speed700mb,wind_speed650mb,wind_speed600mb,wind_speed550mb,wind_speed500mb,wind_speed450mb,wind_speed400mb,wind_speed350mb,wind_speed300mb,wind_speed250mb,wind_speed200mb,wind_direction950mb,wind_direction900mb,wind_direction850mb,wind_direction800mb,wind_direction750mb,wind_direction700mb,wind_direction650mb,wind_direction600mb,wind_direction550mb,wind_direction500mb,wind_direction450mb,wind_direction400mb,wind_direction350mb,wind_direction300mb,wind_direction250mb,wind_direction200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327,-9999.0,-152.734375,4.015625,4.0625,4.0625,4.109375,4.265625,4.625,4.984375,5.328125,5.828125,6.359375,7.046875,7.78125,8.78125,9.296875,-9999.0,53.296875,229.453125,241.25,248.125,256.5625,261.953125,274.765625,273.125,271.796875,268.4375,266.484375,259.53125,264.921875,264.21875,262.65625,-9999.0,-148.375,6.8125,5.3125,5.46875,5.515625,5.75,6.25,6.8125,7.078125,7.4375,9.140625,10.25,10.0625,6.265625,2.515625,31.5,97.5,6.703125,-9999.0,-9999.0,-9999.0,2.28125,13.34375,9.28125,1009.59375,0.796875,-468.09375,210.46875,2.71875
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860,-9999.0,-152.734375,4.015625,4.0625,4.0625,4.109375,4.265625,4.625,4.984375,5.328125,5.828125,6.359375,7.046875,7.78125,8.78125,9.296875,-9999.0,53.296875,229.453125,241.25,248.125,256.5625,261.953125,274.765625,273.125,271.796875,268.4375,266.484375,259.53125,264.921875,264.21875,262.65625,-9999.0,-148.375,6.8125,5.3125,5.46875,5.515625,5.75,6.25,6.8125,7.078125,7.4375,9.140625,10.25,10.0625,6.265625,2.515625,31.5,97.5,6.703125,-9999.0,-9999.0,-9999.0,2.28125,13.34375,9.28125,1009.59375,0.796875,-468.09375,210.46875,2.71875


Guardar los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.

In [22]:
df_total.to_csv("datos/union_clima_medias_tiburones.csv")